# Chapter 24: Triggers (Use Sparingly, Use Well)

Triggers automate actions in response to data changes, ensuring integrity constraints and audit trails execute reliably. However, they introduce hidden execution paths that complicate debugging and can severely impact performance. This chapter establishes when triggers are appropriate, implementation patterns for auditing and cache maintenance, and strict guidelines for avoiding operational nightmares.

## 24.1 Trigger Fundamentals

Triggers bind procedural logic to table events (INSERT, UPDATE, DELETE) with specific timing and granularity.

### 24.1.1 Trigger Types and Timing

```sql
-- Trigger timing determines when the function executes relative to the event:

-- BEFORE: Execute before the operation modifies the table
-- Use for: Validation, modifying values before insertion, preventing operations
CREATE TRIGGER validate_email_trigger
    BEFORE INSERT OR UPDATE ON users
    FOR EACH ROW
    EXECUTE FUNCTION validate_email();

-- AFTER: Execute after the operation completes
-- Use for: Auditing, cascading changes to other tables, cache updates
-- Cannot modify the row being inserted/updated (NEW is read-only in AFTER)
CREATE TRIGGER audit_user_changes
    AFTER INSERT OR UPDATE OR DELETE ON users
    FOR EACH ROW
    EXECUTE FUNCTION audit_user();

-- INSTEAD OF: Execute instead of the operation (views only)
-- Use for: Making views updatable
CREATE TRIGGER update_user_view
    INSTEAD OF UPDATE ON user_summary_view
    FOR EACH ROW
    EXECUTE FUNCTION update_user_via_view();

-- Trigger granularity:
-- FOR EACH ROW: Execute once per affected row (row-level trigger)
-- FOR EACH STATEMENT: Execute once per SQL statement, regardless of rows affected
-- Use statement-level for: Logging statement execution, statistics updates

-- Statement-level example:
CREATE TRIGGER log_table_changes
    AFTER INSERT OR UPDATE OR DELETE ON orders
    FOR EACH STATEMENT
    EXECUTE FUNCTION log_statement_stats();

-- Multiple triggers on same event: Execution order by alphabetical name unless specified
CREATE TRIGGER z_cleanup_trigger  -- Executes last (alphabetically)
    AFTER UPDATE ON users
    FOR EACH ROW
    EXECUTE FUNCTION cleanup_old_data();

CREATE TRIGGER a_validation_trigger  -- Executes first
    BEFORE UPDATE ON users
    FOR EACH ROW
    EXECUTE FUNCTION validate_data();

-- Explicit ordering (PostgreSQL 14+):
CREATE TRIGGER validation_trigger
    BEFORE UPDATE ON users
    FOR EACH ROW
    EXECUTE FUNCTION validate_data();

CREATE TRIGGER enrichment_trigger
    BEFORE UPDATE ON users
    FOR EACH ROW
    AFTER validation_trigger  -- Executes after validation_trigger
    EXECUTE FUNCTION enrich_data();
```

### 24.1.2 The Trigger Function Context

```sql
-- Trigger functions receive special variables automatically:

CREATE OR REPLACE FUNCTION generic_trigger_function()
RETURNS TRIGGER AS $$
BEGIN
    -- TG_NAME: Name of the trigger
    RAISE NOTICE 'Trigger % fired', TG_NAME;
    
    -- TG_OP: Operation that fired trigger (INSERT, UPDATE, DELETE, TRUNCATE)
    RAISE NOTICE 'Operation: %', TG_OP;
    
    -- TG_TABLE_NAME, TG_TABLE_SCHEMA: Target table
    RAISE NOTICE 'Table: %.%', TG_TABLE_SCHEMA, TG_TABLE_NAME;
    
    -- TG_WHEN: BEFORE, AFTER, or INSTEAD OF
    RAISE NOTICE 'Timing: %', TG_WHEN;
    
    -- TG_LEVEL: ROW or STATEMENT
    RAISE NOTICE 'Level: %', TG_LEVEL;
    
    -- NEW: New row values (INSERT, UPDATE). NULL for DELETE
    IF TG_OP = 'INSERT' OR TG_OP = 'UPDATE' THEN
        RAISE NOTICE 'New values: id=%, email=%', NEW.id, NEW.email;
    END IF;
    
    -- OLD: Old row values (UPDATE, DELETE). NULL for INSERT
    IF TG_OP = 'UPDATE' OR TG_OP = 'DELETE' THEN
        RAISE NOTICE 'Old values: id=%, email=%', OLD.id, OLD.email;
    END IF;
    
    -- Return value matters:
    -- BEFORE ROW triggers: Return NULL to abort operation, or NEW to proceed (can modify NEW)
    -- AFTER ROW triggers: Return value ignored, but must return NEW (or NULL for DELETE)
    -- STATEMENT triggers: Must return NULL
    
    IF TG_OP = 'DELETE' THEN
        RETURN OLD;
    ELSE
        RETURN NEW;
    END IF;
END;
$$ LANGUAGE plpgsql;
```

## 24.2 Auditing Implementation Patterns

Audit trails are a legitimate trigger use case, capturing historical data changes for compliance and debugging.

### 24.2.1 Basic Audit Trail

```sql
-- Audit table structure (generic, works for any table):
CREATE TABLE audit_log (
    audit_id BIGSERIAL PRIMARY KEY,
    table_name TEXT NOT NULL,
    operation TEXT NOT NULL CHECK (operation IN ('INSERT', 'UPDATE', 'DELETE')),
    old_data JSONB,
    new_data JSONB,
    changed_at TIMESTAMPTZ DEFAULT NOW(),
    changed_by TEXT DEFAULT CURRENT_USER,
    transaction_id BIGINT DEFAULT txid_current()
);

-- Generic audit trigger function:
CREATE OR REPLACE FUNCTION audit_trigger_function()
RETURNS TRIGGER AS $$
BEGIN
    IF TG_OP = 'INSERT' THEN
        INSERT INTO audit_log (table_name, operation, new_data, changed_by)
        VALUES (TG_TABLE_NAME, 'INSERT', to_jsonb(NEW), CURRENT_USER);
        RETURN NEW;
        
    ELSIF TG_OP = 'UPDATE' THEN
        INSERT INTO audit_log (table_name, operation, old_data, new_data, changed_by)
        VALUES (TG_TABLE_NAME, 'UPDATE', to_jsonb(OLD), to_jsonb(NEW), CURRENT_USER);
        RETURN NEW;
        
    ELSIF TG_OP = 'DELETE' THEN
        INSERT INTO audit_log (table_name, operation, old_data, changed_by)
        VALUES (TG_TABLE_NAME, 'DELETE', to_jsonb(OLD), CURRENT_USER);
        RETURN OLD;
    END IF;
    
    RETURN NULL; -- Should never reach here
END;
$$ LANGUAGE plpgsql SECURITY DEFINER;

-- Apply to sensitive tables:
CREATE TRIGGER users_audit_trigger
    AFTER INSERT OR UPDATE OR DELETE ON users
    FOR EACH ROW
    EXECUTE FUNCTION audit_trigger_function();

-- Querying audit trail:
-- Show history for specific user:
SELECT * FROM audit_log 
WHERE table_name = 'users' 
  AND (new_data->>'id' = '123' OR old_data->>'id' = '123')
ORDER BY changed_at DESC;

-- Detect specific value changes:
SELECT * FROM audit_log 
WHERE table_name = 'users' 
  AND old_data->>'email' != new_data->>'email'
ORDER BY changed_at;
```

### 24.2.2 Selective Auditing (Performance Optimization)

```sql
-- Full JSONB audit is expensive. Audit only specific columns or conditions:

CREATE OR REPLACE FUNCTION selective_audit_function()
RETURNS TRIGGER AS $$
DECLARE
    old_val TEXT;
    new_val TEXT;
BEGIN
    -- Only audit if status changed (ignore other updates)
    IF TG_OP = 'UPDATE' THEN
        IF OLD.status IS DISTINCT FROM NEW.status THEN
            INSERT INTO audit_log (table_name, operation, old_data, new_data)
            VALUES (
                TG_TABLE_NAME, 
                'STATUS_CHANGE',
                jsonb_build_object('status', OLD.status, 'id', OLD.id),
                jsonb_build_object('status', NEW.status, 'id', NEW.id)
            );
        END IF;
        RETURN NEW;
        
    ELSIF TG_OP = 'DELETE' THEN
        INSERT INTO audit_log (table_name, operation, old_data)
        VALUES (TG_TABLE_NAME, 'DELETE', jsonb_build_object('id', OLD.id, 'email', OLD.email));
        RETURN OLD;
    END IF;
    
    RETURN NEW;
END;
$$ LANGUAGE plpgsql;

-- Conditional trigger (PostgreSQL 9.0+):
CREATE TRIGGER audit_status_changes
    AFTER UPDATE ON orders
    FOR EACH ROW
    WHEN (OLD.status IS DISTINCT FROM NEW.status)  -- Only fire if status changed
    EXECUTE FUNCTION log_status_change();
```

## 24.3 Denormalized Cache Maintenance

Triggers can maintain derived data (counts, aggregates, materialized caches) but introduce write amplification.

### 24.3.1 Counter Cache Pattern

```sql
-- Scenario: Orders table is huge, counting pending orders is slow
-- Solution: Maintain counter table with triggers

CREATE TABLE order_stats (
    status TEXT PRIMARY KEY,
    count BIGINT DEFAULT 0
);

-- Initialize:
INSERT INTO order_stats (status, count) 
SELECT status, COUNT(*) FROM orders GROUP BY status;

-- Trigger to maintain counts:
CREATE OR REPLACE FUNCTION update_order_stats()
RETURNS TRIGGER AS $$
BEGIN
    IF TG_OP = 'INSERT' THEN
        UPDATE order_stats SET count = count + 1 WHERE status = NEW.status;
        IF NOT FOUND THEN
            INSERT INTO order_stats (status, count) VALUES (NEW.status, 1);
        END IF;
        RETURN NEW;
        
    ELSIF TG_OP = 'UPDATE' AND OLD.status IS DISTINCT FROM NEW.status THEN
        -- Decrement old status
        UPDATE order_stats SET count = count - 1 WHERE status = OLD.status;
        -- Increment new status
        UPDATE order_stats SET count = count + 1 WHERE status = NEW.status;
        IF NOT FOUND THEN
            INSERT INTO order_stats (status, count) VALUES (NEW.status, 1);
        END IF;
        RETURN NEW;
        
    ELSIF TG_OP = 'DELETE' THEN
        UPDATE order_stats SET count = count - 1 WHERE status = OLD.status;
        RETURN OLD;
    END IF;
    
    RETURN NULL;
END;
$$ LANGUAGE plpgsql;

CREATE TRIGGER maintain_order_stats
    AFTER INSERT OR UPDATE OR DELETE ON orders
    FOR EACH ROW
    EXECUTE FUNCTION update_order_stats();

-- Trade-offs:
-- Pros: SELECT count(*) from order_stats WHERE status = 'pending' is instant (O(1))
-- Cons: Every INSERT/UPDATE/DELETE now does 2 writes (table + counter), doubling write latency
-- Alternative: Periodically refresh materialized view if slight staleness acceptable
```

### 24.3.2 Parent Table Aggregation

```sql
-- Maintain denormalized totals on parent table (anti-pattern warning):

-- Anti-pattern: Storing aggregates that can be calculated
-- Only use if read performance critical and writes infrequent

CREATE TABLE customers (
    customer_id SERIAL PRIMARY KEY,
    email TEXT,
    total_orders INTEGER DEFAULT 0,  -- Denormalized
    total_spent DECIMAL(10,2) DEFAULT 0  -- Denormalized
);

CREATE OR REPLACE FUNCTION update_customer_totals()
RETURNS TRIGGER AS $$
BEGIN
    IF TG_OP = 'INSERT' THEN
        UPDATE customers 
        SET total_orders = total_orders + 1,
            total_spent = total_spent + NEW.total
        WHERE customer_id = NEW.customer_id;
        RETURN NEW;
        
    ELSIF TG_OP = 'DELETE' THEN
        UPDATE customers 
        SET total_orders = total_orders - 1,
            total_spent = total_spent - OLD.total
        WHERE customer_id = OLD.customer_id;
        RETURN OLD;
        
    ELSIF TG_OP = 'UPDATE' AND NEW.customer_id = OLD.customer_id THEN
        -- Same customer, amount changed
        UPDATE customers 
        SET total_spent = total_spent - OLD.total + NEW.total
        WHERE customer_id = NEW.customer_id;
        RETURN NEW;
        
    ELSIF TG_OP = 'UPDATE' AND NEW.customer_id != OLD.customer_id THEN
        -- Customer changed (rare, but handle it)
        UPDATE customers SET total_orders = total_orders - 1, total_spent = total_spent - OLD.total 
        WHERE customer_id = OLD.customer_id;
        UPDATE customers SET total_orders = total_orders + 1, total_spent = total_spent + NEW.total 
        WHERE customer_id = NEW.customer_id;
        RETURN NEW;
    END IF;
    
    RETURN NULL;
END;
$$ LANGUAGE plpgsql;

CREATE TRIGGER update_customer_stats
    AFTER INSERT OR UPDATE OR DELETE ON orders
    FOR EACH ROW
    EXECUTE FUNCTION update_customer_totals();

-- WARNING: This pattern creates tight coupling. If trigger fails, order insertion fails.
-- Consider: Asynchronous update via queue, or calculate on read with caching layer
```

## 24.4 Validation and Complex Constraints

Triggers enforce constraints too complex for CHECK or FOREIGN KEY constraints.

### 24.4.1 Cross-Table Validation

```sql
-- Enforce: Cannot create order if customer status is 'suspended'
-- (Cannot use FK because condition is more complex than existence)

CREATE OR REPLACE FUNCTION check_customer_status()
RETURNS TRIGGER AS $$
DECLARE
    cust_status TEXT;
BEGIN
    SELECT status INTO cust_status 
    FROM customers 
    WHERE customer_id = NEW.customer_id;
    
    IF cust_status = 'suspended' THEN
        RAISE EXCEPTION 'Cannot create order for suspended customer %', NEW.customer_id
            USING ERRCODE = 'check_violation',
                  HINT = 'Reactivate customer account first';
    END IF;
    
    RETURN NEW;
END;
$$ LANGUAGE plpgsql;

CREATE TRIGGER validate_customer_before_order
    BEFORE INSERT OR UPDATE ON orders
    FOR EACH ROW
    EXECUTE FUNCTION check_customer_status();

-- Note: Race condition possible!
-- If customer is suspended between trigger check and order commit, 
-- constraint is violated. Use SERIALIZABLE isolation or lock customer row:

-- Safer version with locking:
CREATE OR REPLACE FUNCTION check_customer_status_safe()
RETURNS TRIGGER AS $$
DECLARE
    cust_status TEXT;
BEGIN
    -- Lock customer row (ensures status doesn't change during transaction)
    SELECT status INTO cust_status 
    FROM customers 
    WHERE customer_id = NEW.customer_id
    FOR SHARE;  -- Prevents status update until order commits
    
    IF cust_status = 'suspended' THEN
        RAISE EXCEPTION 'Customer % is suspended', NEW.customer_id;
    END IF;
    
    RETURN NEW;
END;
$$ LANGUAGE plpgsql;
```

### 24.4.2 Temporal Constraints

```sql
-- Ensure no overlapping date ranges for resource bookings

CREATE OR REPLACE FUNCTION check_booking_overlap()
RETURNS TRIGGER AS $$
DECLARE
    overlapping INTEGER;
BEGIN
    SELECT COUNT(*) INTO overlapping
    FROM bookings
    WHERE resource_id = NEW.resource_id
      AND id IS DISTINCT FROM NEW.id  -- Exclude current row on updates
      AND (NEW.start_time, NEW.end_time) OVERLAPS (start_time, end_time);
    
    IF overlapping > 0 THEN
        RAISE EXCEPTION 'Booking overlaps with existing reservation for resource %', NEW.resource_id
            USING ERRCODE = 'exclusion_violation';
    END IF;
    
    RETURN NEW;
END;
$$ LANGUAGE plpgsql;

CREATE TRIGGER prevent_booking_overlap
    BEFORE INSERT OR UPDATE ON bookings
    FOR EACH ROW
    EXECUTE FUNCTION check_booking_overlap();

-- Alternative: Use EXCLUDE constraint (declarative, faster, no trigger needed):
-- ALTER TABLE bookings ADD CONSTRAINT no_overlap 
-- EXCLUDE USING gist (resource_id WITH =, tsrange(start_time, end_time) WITH &&);
-- Requires btree_gist extension, but is superior to trigger approach
```

## 24.5 Hidden Complexity and Risks

Triggers create "action at a distance" that violates the principle of least surprise and complicates debugging.

### 24.5.1 The Debugging Nightmare

```sql
-- Problem: Operations have invisible side effects
UPDATE users SET status = 'active' WHERE id = 123;
-- This seemingly simple statement might:
-- 1. Insert audit rows
-- 2. Update 5 counter tables
-- 3. Send notifications via pg_notify
-- 4. Call external HTTP endpoints (via extensions)
-- 5. Fail due to trigger bug in unrelated table (cascading triggers)

-- Debugging triggers:
-- 1. Check pg_trigger catalog:
SELECT 
    tgname,
    tgrelid::regclass,
    CASE tgtype & 66 
        WHEN 2 THEN 'BEFORE'
        WHEN 64 THEN 'INSTEAD OF'
        ELSE 'AFTER'
    END as timing,
    CASE tgtype & 28
        WHEN 4 THEN 'INSERT'
        WHEN 8 THEN 'DELETE'
        WHEN 16 THEN 'UPDATE'
        WHEN 28 THEN 'INSERT/DELETE/UPDATE'
    END as events,
    proname as function_name
FROM pg_trigger t
JOIN pg_proc p ON t.tgfoid = p.oid
WHERE tgrelid = 'users'::regclass
  AND NOT tgisinternal;

-- 2. Enable trigger debugging:
SET plpgsql.extra_errors TO 'all';  -- Stricter checking
-- Or add RAISE NOTICE at trigger entry/exit

-- 3. Session-level trigger disabling (emergency only):
ALTER TABLE users DISABLE TRIGGER ALL;
-- Or specific trigger:
ALTER TABLE users DISABLE TRIGGER audit_user_changes;
-- Remember to re-enable!
```

### 24.5.2 Cascading Trigger Storms

```sql
-- Anti-pattern: Chain reactions
-- Table A trigger updates Table B
-- Table B trigger updates Table C
-- Table C trigger updates Table A (circular) or fires unrelated actions

-- Example of cascading write amplification:
-- Insert 1 row into orders
-- -> Trigger updates customers (1 write)
--    -> Trigger on customers updates customer_stats (1 write)
--       -> Trigger on customer_stats logs to audit (1 write)
-- 1 insert becomes 4 writes, 3x latency increase

-- Prevention:
-- 1. Document trigger dependencies (draw data flow diagrams)
-- 2. Use session variables to prevent recursion:
CREATE OR REPLACE FUNCTION safe_trigger()
RETURNS TRIGGER AS $$
BEGIN
    -- Prevent recursive firing
    IF current_setting('myapp.trigger_depth', true)::int > 0 THEN
        RETURN NEW;
    END IF;
    
    PERFORM set_config('myapp.trigger_depth', 
        (current_setting('myapp.trigger_depth', true)::int + 1)::text, true);
    
    -- Do work...
    
    PERFORM set_config('myapp.trigger_depth', 
        (current_setting('myapp.trigger_depth', true)::int - 1)::text, true);
    RETURN NEW;
END;
$$ LANGUAGE plpgsql;

-- 3. Prefer application-level logic for complex workflows (easier to trace)
```

### 24.5.3 Performance Degradation

```sql
-- Triggers execute in the same transaction as the firing statement
-- Slow triggers block the application:

-- Bad: Synchronous HTTP call in trigger
CREATE OR REPLACE FUNCTION notify_external_service()
RETURNS TRIGGER AS $$
BEGIN
    -- NEVER DO THIS: HTTP request in trigger
    -- PERFORM http_get('https://api.example.com/notify?user=' || NEW.id);
    -- Network latency (500ms) added to every INSERT
    -- If API is down, INSERT fails (transaction rolls back)
    
    -- Better: Asynchronous notification
    PERFORM pg_notify('user_changes', json_build_object('id', NEW.id, 'email', NEW.email)::text);
    -- Application listens and makes HTTP call asynchronously
    
    RETURN NEW;
END;
$$ LANGUAGE plpgsql;

-- Trigger execution statistics (PostgreSQL 14+):
SELECT 
    tgname,
    tbl.relname as table_name,
    ps.calls,
    ps.total_exec_time,
    ps.mean_exec_time
FROM pg_stat_user_functions ps
JOIN pg_proc p ON ps.funcid = p.oid
JOIN pg_trigger t ON p.oid = t.tgfoid
JOIN pg_class tbl ON t.tgrelid = tbl.oid
ORDER BY ps.total_exec_time DESC;

-- If trigger mean_exec_time > 1ms on high-volume tables, investigate
```

## 24.6 Testing Trigger Behavior

Triggers require specific testing strategies to verify they fire correctly without side effects.

### 24.6.1 Unit Testing Triggers

```sql
-- Testing framework approach (using pgTAP or similar):

-- 1. Setup test data
BEGIN;
    INSERT INTO customers (id, email, status) VALUES (999, 'test@test.com', 'suspended');
    
    -- 2. Attempt operation that should fail
    DO $$
    BEGIN
        INSERT INTO orders (customer_id, total) VALUES (999, 100);
        RAISE EXCEPTION 'Should have failed with suspended customer';
    EXCEPTION
        WHEN check_violation THEN
            RAISE NOTICE 'Correctly rejected order for suspended customer';
    END;
    $$;
    
    -- 3. Verify audit trail created
    ASSERT (SELECT COUNT(*) FROM audit_log WHERE table_name = 'customers') = 1;
    
    -- 4. Verify counters updated
    UPDATE customers SET status = 'active' WHERE id = 999;
    INSERT INTO orders (customer_id, total) VALUES (999, 100);
    ASSERT (SELECT total_orders FROM customers WHERE id = 999) = 1;
    
ROLLBACK;  -- Clean up test data

-- Isolated trigger testing (disable then enable):
-- ALTER TABLE orders DISABLE TRIGGER audit_trigger;
-- Run performance test (measure without trigger overhead)
-- ALTER TABLE orders ENABLE TRIGGER audit_trigger;
```

### 24.6.2 Trigger Inspection

```sql
-- Verify trigger firing order:
CREATE OR REPLACE FUNCTION log_trigger_execution()
RETURNS TRIGGER AS $$
BEGIN
    RAISE NOTICE 'Trigger % fired on % for %', TG_NAME, TG_TABLE_NAME, TG_OP;
    IF TG_OP = 'DELETE' THEN
        RETURN OLD;
    ELSE
        RETURN NEW;
    END IF;
END;
$$ LANGUAGE plpgsql;

-- Attach to table to debug firing sequence:
CREATE TRIGGER zzz_debug_trigger
    BEFORE INSERT OR UPDATE OR DELETE ON target_table
    FOR EACH ROW
    EXECUTE FUNCTION log_trigger_execution();
```

## 24.7 Best Practices and Anti-Patterns

### 24.7.1 When to Use Triggers

```sql
-- APPROPRIATE uses:
-- 1. Auditing: Immutable history tracking (who changed what when)
-- 2. Cross-database replication: Logical decoding output
-- 3. Complex constraints: Temporal ranges, exclusion constraints that can't use EXCLUDE
-- 4. Immutable calculated columns: Set once on insert (prefer GENERATED ALWAYS in PostgreSQL 12+)

-- PostgreSQL 12+ alternative for calculated columns:
CREATE TABLE products (
    price DECIMAL(10,2),
    tax_rate DECIMAL(4,4),
    total_price DECIMAL(10,2) GENERATED ALWAYS AS (price * (1 + tax_rate)) STORED
);
-- No trigger needed, constraint is declarative and enforced at storage layer

-- INAPPROPRIATE uses (do in application instead):
-- 1. Business logic validation (hard to debug, test)
-- 2. External API calls (blocks transactions, reliability issues)
-- 3. Sending emails/notifications (async job queue better)
-- 4. Simple defaults (use DEFAULT clause)
-- 5. Data formatting (cleanse in application before INSERT)
```

### 24.7.2 Trigger Safety Checklist

```sql
-- Before creating a trigger, verify:

-- 1. Idempotency: Can trigger run twice safely?
CREATE OR REPLACE FUNCTION idempotent_audit()
RETURNS TRIGGER AS $$
BEGIN
    -- Check if already audited this transaction (prevent duplicates on retries)
    IF EXISTS (SELECT 1 FROM audit_log WHERE transaction_id = txid_current() 
               AND table_name = TG_TABLE_NAME 
               AND (new_data->>'id')::bigint = NEW.id) THEN
        RETURN NEW;
    END IF;
    
    INSERT INTO audit_log (...) VALUES (...);
    RETURN NEW;
END;
$$ LANGUAGE plpgsql;

-- 2. Null safety: Handle NULL values explicitly
IF NEW.status IS DISTINCT FROM OLD.status THEN  -- Use IS DISTINCT FROM (handles NULLs)
    -- Do work
END IF;

-- 3. Performance: Keep trigger execution < 1ms for high-volume tables
-- Avoid: Complex queries, joins to large tables, sequential scans

-- 4. Transaction safety: Never COMMIT/ROLLBACK inside trigger
-- (Only procedures can do this, not trigger functions)

-- 5. Security: Use SECURITY DEFINER only if necessary, with safe search_path
CREATE TRIGGER audit_trigger
    AFTER INSERT ON sensitive_table
    FOR EACH ROW
    EXECUTE FUNCTION audit_function();  -- Should be SECURITY INVOKER usually
```

---

## Chapter Summary

In this chapter, you learned:

1. **Trigger Types**: BEFORE triggers modify data or validate before persistence; AFTER triggers handle cascading logic and auditing (NEW is read-only); INSTEAD OF enables updatable views. Row-level triggers execute per row; statement-level triggers execute once per operation.

2. **Auditing**: Use JSONB columns to store flexible row snapshots in audit tables. Prefer selective auditing (specific columns or status changes) over full-row logging to reduce write overhead. Use `txid_current()` to correlate audit entries with transactions.

3. **Cache Maintenance**: Counter caches and denormalized aggregates in triggers provide fast reads at the cost of doubled write latency and tight coupling. Consider materialized views or application-level cache invalidation queues for less critical data.

4. **Validation**: Triggers enforce cross-table constraints and temporal logic that FOREIGN KEY and CHECK constraints cannot express. Use `FOR SHARE` row locking in validation triggers to prevent race conditions between check and commit.

5. **Hidden Complexity**: Triggers create "spooky action at a distance" where simple UPDATE statements cascade into multiple table modifications, complicating debugging and performance analysis. Document all trigger chains and avoid circular dependencies.

6. **Testing**: Test triggers within transactions that roll back to isolate side effects. Use `RAISE NOTICE` or logging triggers to verify execution order. Monitor `pg_stat_user_functions` to identify slow triggers blocking high-volume operations.

**Next:** In Chapter 25, we will explore Extensions and Ecosystem—covering extension management, trusted vs untrusted extensions, versioning strategies, and the operational implications of incorporating external code into your database.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='23. plpgsql_essentials.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='25. extensions_and_ecosystem.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
